In [149]:
import duckdb
import pandas as pd
from statsbombpy import sb

# DuckDB bağlantısı
#con = duckdb.connect("bitirme_test.db")


In [150]:
def get_homeTeam(data):
    team = data["home_team"].unique()
    return team

In [151]:
def pass_data_creation(events_extra_info,match_ID):

    events_df = sb.events(match_id=match_ID)
    events_df = events_df.set_index("match_id").join(
        events_extra_info.set_index("match_id"), how="left", rsuffix='_match'
    ).reset_index()

    # Koordinatları ayır
    events_df[['x', 'y']] = events_df['location'].apply(pd.Series)
    events_df[['pass_end_x', 'pass_end_y']] = events_df['pass_end_location'].apply(pd.Series)

    # Paslar tablosunu oluştur
    passes = events_df[(events_df.type == "Pass")]
    passes = passes[passes["pass_outcome"] != "Injury Clearance"]
    shots = events_df[(events_df.type == "Shot")][["player","shot_type","shot_key_pass_id","shot_outcome","shot_statsbomb_xg"]].dropna()
    passes_M = passes.set_index("id").join(
        shots.set_index("shot_key_pass_id"), how="left", rsuffix='_shotFromPass'
    )
    passes_M = passes_M[passes_M["pass_outcome"] != "Injury Clearance"]

    passes_M["pass_succes"] = passes_M.pass_outcome.isna()
    passes_M["f3rd_pass"] = (passes_M.x < 80) & (passes_M.pass_end_x > 80)
    passes_M["pass_progression"] = passes_M["x"] - passes_M["pass_end_x"]
    passes_M.loc[passes_M.pass_outcome.notna(), "pass_progression"] = None
    passes_M["shot_assisted_pass"] = passes_M.pass_outcome.isna() &  passes_M["shot_outcome_shotFromPass"].notna()
    passes_M["goal_assisted_pass"] = passes_M.pass_outcome.isna() & passes_M["shot_outcome_shotFromPass"] == "Goal"
    passes_M["cross_pass"] = (passes_M.pass_cross == True)
    passes_M["box_pass"] = (
        (passes_M.x < 102) & ((passes_M.y < 18) | (passes_M.y > 62)) &
        (passes_M.pass_end_x > 102) & ((passes_M.pass_end_y > 18) | (passes_M.pass_end_y < 62))
    )

    passes_M["rowID"] = passes_M.index

    return passes_M

In [152]:
def foul_data_creation(events_extra_info,match_ID):

    events_df = sb.events(match_id=match_ID)
    events_df = events_df.set_index("match_id").join(
        events_extra_info.set_index("match_id"), how="left", rsuffix='_match'
    ).reset_index()

    # Koordinatları ayır
    events_df[['x', 'y']] = events_df['location'].apply(pd.Series)
    events_df[['pass_end_x', 'pass_end_y']] = events_df['pass_end_location'].apply(pd.Series)

    # Fauller tablosunu oluştur
    foul_commited = events_df[(events_df.type == "Foul Committed")][
        ["id", "type", "player", "related_events", "foul_committed_advantage", "foul_committed_card", "x", "y"]
    ].set_index("id")
    fouls = events_df[(events_df.type == "Foul Won")].explode("related_events")
    fouls_M = fouls.set_index("related_events").join(foul_commited, how="left", rsuffix='_foul_commitedJ')
    fouls_M = fouls_M[fouls_M["player_foul_commitedJ"].notna()]
    fouls_M["rowID"] = fouls_M.index
    return fouls_M

In [153]:
matches = sb.matches(competition_id=11, season_id=27)
events_extra_info = matches[["match_id","season","competition","home_team","away_team","referee"]]


/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [155]:
passDF = pass_data_creation(events_extra_info,3825838)

/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [156]:
passDF[["match_id","season","competition","home_team","away_team","referee",
    "player",
    "x",
    "y",
    "pass_end_x",
    "pass_end_y",
    "player_shotFromPass",
    "shot_outcome_shotFromPass",
    "shot_statsbomb_xg_shotFromPass",
    "pass_succes",
    "f3rd_pass",
    "pass_progression",
    "shot_assisted_pass",
    "goal_assisted_pass",
    "cross_pass",
    "box_pass"
]].head(10)

,match_id,season,competition,home_team,away_team,referee,player,x,y,pass_end_x,...,player_shotFromPass,shot_outcome_shotFromPass,shot_statsbomb_xg_shotFromPass,pass_succes,f3rd_pass,pass_progression,shot_assisted_pass,goal_assisted_pass,cross_pass,box_pass
id,,,,,,,,,,,,,,,,,,,,,
972873a7-3d00-4f4a-8d9a-cc429f3eb973,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Saúl Ñíguez Esclapez,61.0,40.1,60.3,...,NaN,NaN,NaN,True,False,0.7,False,False,False,False
e4c05258-39a5-4c9c-97f7-6ebc0b5b9d85,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Thomas Teye Partey,60.6,42.9,45.4,...,NaN,NaN,NaN,True,False,15.2,False,False,False,False
92a1865e-d479-4c03-937e-f88b60123cd8,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Jorge Resurrección Merodio,46.2,43.1,45.0,...,NaN,NaN,NaN,True,False,1.2,False,False,False,False
768060eb-6a53-4c72-aaba-8ef65cae7ac9,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Filipe Luís Kasmirski,45.9,14.9,92.5,...,NaN,NaN,NaN,False,True,NaN,False,False,False,False
f148b397-1564-4331-83ac-d173330a0953,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Gorka Iraizoz Moreno,28.4,51.3,41.5,...,NaN,NaN,NaN,True,False,-13.1,False,False,False,False
6bf4307b-6c1d-467e-b770-bb75f2e6d83a,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Xabier Etxeita Gorritxategi,44.0,20.2,39.5,...,NaN,NaN,NaN,True,False,4.5,False,False,False,False
7c84e0c2-177b-444f-848e-1e1e3558be02,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Eneko Bóveda Altube,39.3,53.1,91.1,...,NaN,NaN,NaN,False,True,NaN,False,False,False,False
4bf6bb6d-633d-48b5-b225-6b6b36041f6c,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Filipe Luís Kasmirski,29.0,11.0,42.6,...,NaN,NaN,NaN,True,False,-13.6,False,False,False,False
ebbe304c-039d-4f3b-9086-27c10074729b,3825838,2015/2016,Spain - La Liga,Athletic Club,Atlético Madrid,Juan Martínez Munuera,Saúl Ñíguez Esclapez,42.2,15.7,28.4,...,NaN,NaN,NaN,True,False,13.8,False,False,False,False


In [140]:
print(get_homeTeam(passDF))


['Atlético Madrid']


In [141]:
foulsDF = foul_data_creation(events_extra_info,3825882)

/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [142]:

#con.register("passes",passes_M)
#con.register("fouls",fouls_M)

In [143]:
#con.table("passes").show()


In [146]:
matches = sb.matches(competition_id=11, season_id=27)
events_extra_info = matches[["match_id","season","competition","home_team","away_team","referee"]]


for match_ID in [3825848,3825895]:

    passData = pass_data_creation(events_extra_info,match_ID)

    foulData = foul_data_creation(events_extra_info,match_ID)

    passDF = pd.concat([passDF, passData], axis=0, ignore_index=True)
    foulsDF = pd.concat([foulsDF, ], axis=0, ignore_index=True)




/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/emirysaglam/GitHub/Bitirme/bitirme-venv/lib/python3.9/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [147]:
passDF[["match_id","season","competition","home_team","away_team","referee",
    "player",
    "x",
    "y",
    "pass_end_x",
    "pass_end_y",
    "player_shotFromPass",
    "shot_outcome_shotFromPass",
    "shot_statsbomb_xg_shotFromPass",
    "pass_succes",
    "f3rd_pass",
    "pass_progression",
    "shot_assisted_pass",
    "goal_assisted_pass",
    "cross_pass",
    "box_pass"
]].tail(10)

,match_id,season,competition,home_team,away_team,referee,player,x,y,pass_end_x,...,player_shotFromPass,shot_outcome_shotFromPass,shot_statsbomb_xg_shotFromPass,pass_succes,f3rd_pass,pass_progression,shot_assisted_pass,goal_assisted_pass,cross_pass,box_pass
2880,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,David Simón Rodríguez Santana,29.0,75.1,46.9,...,NaN,NaN,NaN,True,False,-17.9,False,False,False,False
2881,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Antolín Alcáraz Viveros,66.6,77.2,61.0,...,NaN,NaN,NaN,True,False,5.6,False,False,False,False
2882,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Marco Andreolli,17.7,51.6,35.2,...,NaN,NaN,NaN,True,False,-17.5,False,False,False,False
2883,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Víctor Machín Pérez,60.3,33.5,72.6,...,NaN,NaN,NaN,True,False,-12.3,False,False,False,False
2884,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Kevin Gameiro,82.1,39.3,94.0,...,NaN,NaN,NaN,True,False,-11.9,False,False,False,False
2885,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Javier Varas Herrera,6.0,56.9,68.8,...,NaN,NaN,NaN,False,False,NaN,False,False,False,False
2886,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Timothée Kolodziejczak,51.3,24.7,57.0,...,NaN,NaN,NaN,False,False,NaN,False,False,False,False
2887,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Vicente Iborra De La Fuente,49.8,50.4,37.5,...,NaN,NaN,NaN,True,False,12.3,False,False,False,False
2888,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Timothée Kolodziejczak,37.9,35.4,50.4,...,NaN,NaN,NaN,True,False,-12.5,False,False,False,False
2889,3825895,2015/2016,Spain - La Liga,Las Palmas,Sevilla,Carlos del Cerro Grande,Benoît Trémoulinas,53.5,10.5,77.4,...,NaN,NaN,NaN,True,False,-23.9,False,False,False,False


In [148]:
passDF["away_team"].unique()

array(['Celta Vigo', 'Eibar', 'Sevilla'], dtype=object)